<a href="https://colab.research.google.com/github/Zimo0528/Supply-Chain/blob/main/Supply_Chain_Tailored_Sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Given data for the four products
products = {
    1: {'demand': 1000, 'common_order_cost': 100, 'specific_order_cost': 10, 'unit_cost': 50, 'holding_cost_rate': 0.2},
    2: {'demand': 300, 'common_order_cost': 100, 'specific_order_cost': 20, 'unit_cost': 60, 'holding_cost_rate': 0.2},
    3: {'demand': 100, 'common_order_cost': 100, 'specific_order_cost': 25, 'unit_cost': 30, 'holding_cost_rate': 0.2},
    4: {'demand': 50, 'common_order_cost': 100, 'specific_order_cost': 25, 'unit_cost': 30, 'holding_cost_rate': 0.2}
}

# Function to calculate EOQ and total cost for independent sourcing
def calculate_costs(product_info):
    D = product_info['demand']
    S = product_info['common_order_cost'] + product_info['specific_order_cost']
    H = product_info['holding_cost_rate'] * product_info['unit_cost']
    EOQ = np.sqrt((2 * D * S) / H)
    total_cost = (D / EOQ) * S + (EOQ / 2) * H
    return EOQ, total_cost

# Calculate and print the costs for independent sourcing
total_annual_cost = 0
for product in products.values():
    EOQ, cost = calculate_costs(product)
    total_annual_cost += cost
    print(f"EOQ for product with demand {product['demand']}: {EOQ:.2f}, Total Cost: {cost:.2f}")

print(f"Total annual operational cost for independent sourcing: {total_annual_cost:.2f}")


EOQ for product with demand 1000: 148.32, Total Cost: 1483.24
EOQ for product with demand 300: 77.46, Total Cost: 929.52
EOQ for product with demand 100: 64.55, Total Cost: 387.30
EOQ for product with demand 50: 45.64, Total Cost: 273.86
Total annual operational cost for independent sourcing: 3073.92


In [6]:
# Sum of specific order costs
total_specific_order_cost = sum(product['specific_order_cost'] for product in products.values())

# Common order cost (incurred once because of combined sourcing)
common_order_cost = products[1]['common_order_cost']  # Assuming all products share this cost

# Calculate combined EOQ
S = common_order_cost + total_specific_order_cost  # Total ordering cost
n = np.sqrt(sum(product['demand'] * product['holding_cost_rate'] * product['unit_cost'] for product in products.values())/ (2 * S))

def calculate_costs(product_info):
    D = product_info['demand']
    H = product_info['holding_cost_rate'] * product_info['unit_cost']
    holding_cost = (D / n / 2) * H
    return holding_cost

total_holding_cost = 0
for product in products.values():
    cost = calculate_costs(product)
    total_holding_cost += cost
    print(f"Total holding cost: {cost:.2f}")
total_annual_cost = S * n + total_holding_cost

print(f"Total Cost: {total_annual_cost:.2f}")

Total holding cost: 787.84
Total holding cost: 283.62
Total holding cost: 47.27
Total holding cost: 23.64
Total Cost: 2284.73


In [7]:
import math
def calculate_freq(product_info):
    D = product_info['demand']
    S = product_info['common_order_cost'] + product_info['specific_order_cost']
    H = product_info['holding_cost_rate'] * product_info['unit_cost']
    EOQ = np.sqrt((2 * D * S) / H)
    frequency = D / EOQ
    return frequency

def calculate_new_freq(product_info):
    D = product_info['demand']
    S = product_info['specific_order_cost']
    H = product_info['holding_cost_rate'] * product_info['unit_cost']
    Q = np.sqrt((2 * D * S) / H)
    frequency = D / Q
    return frequency

def calculate_costs(product_info):
    D = product_info['demand']
    H = product_info['holding_cost_rate'] * product_info['unit_cost']
    n = product_info['new_freq']
    holding_cost = (D / n / 2) * H
    return holding_cost

# Calculate and print the costs for independent sourcing
frequency = []
for i in range(4):
    freq = calculate_freq(products[i+1])
    products[i+1]['freq'] = freq
    frequency.append(freq)
largest_freq = max(frequency)
index = frequency.index(largest_freq)

for i in range(4):
  if i == index:
    products[i+1]['m'] = 1
  else:
    new_freq = calculate_new_freq(products[i+1])
    products[i+1]['m'] = math.ceil(largest_freq / new_freq)

new_large = np.sqrt(sum(product['demand'] * product['holding_cost_rate'] * product['unit_cost'] * product['m'] for product in products.values())/ (2 * (products[1]['common_order_cost'] + sum(product['specific_order_cost'] / product['m'] for product in products.values()))))

for i in range(4):
  if i == index:
    products[i+1]['new_freq'] = new_large
  else:
    new_freq = calculate_new_freq(products[i+1])
    products[i+1]['new_freq'] = new_large / products[i+1]['m']

S = products[1]['common_order_cost'] + sum(product['specific_order_cost'] / product['m'] for product in products.values())

total_holding_cost = 0
for product in products.values():
    cost = calculate_costs(product)
    total_holding_cost += cost
    print(f"Total Cost: {cost:.2f}")
total_annual_cost = S * new_large + total_holding_cost
print(f"Total Cost: {total_annual_cost:.2f}")

Total Cost: 693.08
Total Cost: 249.51
Total Cost: 83.17
Total Cost: 62.38
Total Cost: 2176.27
